# Spark Exercises

Within your codeup-data-science directory, create a new repo named spark-exercises. This will be where you do your work for this module. Create a repository on GitHub with the same name, and link your local repository to GitHub.

Save this work in your spark-exercises repo. Then add, commit, and push your changes.

Create a jupyter notebook or python script named spark101 for this exercise.

### Imports

In [24]:
import pandas as pd
import numpy as np
#create the spark session
import pyspark
from pyspark.sql.functions import concat, sum, avg, min, max, count, mean
from pyspark.sql.functions import when
from pyspark.sql.functions import month, year, quarter
from pyspark.sql.functions import col, expr
from pyspark.sql.functions import asc, desc
from pyspark.sql.functions import lit
spark = pyspark.sql.SparkSession.builder.getOrCreate()

### 1. Create a spark data frame that contains your favorite programming languages.
- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

#### The name of the column should be language

In [ ]:
df = spark.createDataFrame(
    pd.DataFrame({'language': ['python', 'java', 'r', 'c']}))

In [ ]:
df.show()

#### View the schema of the dataframe

In [ ]:
df.printSchema()

#### Output the shape of the dataframe

- spark does not have a .shape function

In [ ]:
df.count()

In [ ]:
len(df.columns)

In [ ]:
print(df.count(), "rows", len(df.columns), "columns")


#### Show the first 5 records in the dataframe

In [ ]:
df.show(5)

### 2. Load the mpg dataset as a spark dataframe.
- Create 1 column of output that contains a message like the one below (for each vehicle):
The 1999 audi a4 has a 4 cylinder engine.
- Transform the trans column so that it only contains either manual or auto.

#### Create 1 column of output that contains a message like the one below (for each vehicle): 
- The 1999 audi a4 has a 4 cylinder engine.

In [ ]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

In [ ]:
mpg.select(mpg.)

#### Transform the trans column so that it only contains either manual or auto.

In [ ]:

mpg.select(mpg.trans,
          when((mpg.trans.contains('auto')), 'auto')
          .otherwise('manual')
          .alias('trans')).show(20)

### 3. Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [ ]:
from pydataset import data
pandas_dataframe = data("tips")
df = spark.createDataFrame(pandas_dataframe)
df.show()

#### What percentage of observations are smokers?

In [ ]:
df.where(df.smoker == 'Yes').count() / df.count() * 100

#### Create a column that contains the tip percentage

In [ ]:
col = (col('tip') / col('total_bill'))


In [ ]:
df.select('*', col.alias('tip_pct')).show()

#### Calculate the average tip percentage for each combination of sex and smoker.

In [ ]:
#female smoker
df.where(df.sex == 'Female').where(df.smoker == 'Yes').show()

In [ ]:
#female non-smoker
df.where(df.sex == 'Female').where(df.smoker == 'No').show()

In [ ]:
#male smoker
df.where(df.sex == 'Male').where(df.smoker == 'Yes').show()

In [ ]:
#male non-smoker
df.where(df.sex == 'Male').where(df.smoker == 'No').show()

In [ ]:

tips.rollup('sex','smoker').agg(avg(tips.tip_percent)).show()

### 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to farenheight.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [3]:
from vega_datasets import data

df = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
df = spark.createDataFrame(df)
df.show(6)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
|2012-01-06|          2.5|     4.4|     2.2| 2.2|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 6 rows



#### Convert the temperatures to farenheight.

In [ ]:
* (9/5) + 32

#### Which month has the most rain, on average?

In [13]:
df.withColumn('month', month('date')).groupBy('month').agg(sum('precipitation').alias('total_rainfall')).sort(desc('total_rainfall')).show()

+-----+------------------+
|month|    total_rainfall|
+-----+------------------+
|   11|             642.5|
|   12| 622.7000000000002|
|    3|             606.2|
|   10|             503.4|
|    1|465.99999999999994|
|    2|             422.0|
|    4|             375.4|
|    9|235.49999999999997|
|    5|             207.5|
|    8|             163.7|
|    6|             132.9|
|    7|              48.2|
+-----+------------------+



#### Which year was the windiest?

In [15]:
df.withColumn('year', year('date')).groupBy('year').agg(sum('wind').alias('total_wind')).sort(desc('total_wind')).show()

+----+------------------+
|year|        total_wind|
+----+------------------+
|2012|            1244.7|
|2014|1236.5000000000007|
|2015|1153.3000000000002|
|2013|1100.8000000000006|
+----+------------------+



#### What is the most frequent type of weather in January?

In [21]:
(
    df.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



#### What is the average high and low temperature on sunny days in July in 2013 and 2014?

In [25]:
(
    df.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max").alias("average_high_temp"),
        avg("temp_min").alias("average_low_temp"),
    )
    .show()
)

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|26.828846153846158|14.18269230769231|
+------------------+-----------------+



#### What percentage of days were rainy in q3 of 2015?

In [26]:
(
    df.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
    .show()
)

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



#### For each year, find what percentage of days it rained (had non-zero precipitation).

In [27]:
(
    df.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("precipitation") > 0, 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
    .show()
)

+-------------------+
|          avg(rain)|
+-------------------+
|0.18478260869565216|
+-------------------+

